# Catalunya's MOBILITY INDEXES analysis in the COVID-19 restrictions

### Code contributors:
- Julià Vicens <julian.vicens@eurecat.org>
- Jordi Grau <jordi.grau@eurecat.org>

### Content
This notebook analyses the differences in Catalunya's mobility across the previously defined COVID-19 phases using visual analysis. In this notebook the focus are the MITMA MOBILITY INDEXES. 

### Contents

1. [Mobility indexes](#mi)  
    1.1. [Temporal Series](#mi-ts)   
    1.2. [Maps](#mi-maps)      
    1.3. [Heatmaps](#mi-heatmaps)    


2. [Δ Mobility indexes](#Δmi)  
    1.1. [Temporal Series](#Δmi-ts)   
    1.2. [Maps](#Δmi-maps)      
    1.3. [Heatmaps](#Δmi-heatmaps) 

In [ ]:
from src.mobility_context_and_queries import *
from src.mobility_plots import *

In [ ]:
# Get all phases
print("All possible phases:\n")
all_phases = []
for phase in phases_list:
    phase_name, phase_period = list(phase.items())[0]
    print(f'- ({phase_name}, {phase_period["start"]}, {phase_period["end"]})')
    all_phases.append(phase_name)
    
#############################
############ FILL ###########
#############################

# Fill target phases wanted for the study
study_phases = [
    'precovid', 'lockdown', 'mobilitat_essenc', 'fase_0', 'desescalada', 
    'no_restriccions', 'alerta_5_inici', 'alerta_5_tr1', 'alerta_5_tr1_2', 'nadal_cap_reis']

# Print target phases names and dates
phases_dates_targets = []
for phase_dict in phases_list:
    for phase_name, dates in phase_dict.items():
        if phase_name in study_phases:
            phases_dates_targets.append((phase_name, dates['start'], dates['end']))
print('\nStudy about this phases:\n')
for study in phases_dates_targets:
    print(f'- {study}')

In [ ]:
# Get all motivs
print("All possible motivs:\n")
all_motivs = []
for motiv in motiv_list:
    motiv_name, motiv_period = list(motiv.items())[0]
    print(f'- ({motiv_name}, {motiv_period["start"]}, {motiv_period["end"]})')
    all_motivs.append(motiv_name)

#############################
############ FILL ###########
#############################

# Fill target motivs wanted for the study
study_motivs = []

# Print target motivs names and dates
motivs_dates_targets = []
for motiv_dict in motiv_list:
    for motiv_name, dates in motiv_dict.items():
        if motiv_name in study_motivs:
            motivs_dates_targets.append((motiv_name, dates['start'], dates['end']))
print('\nStudy about this motivs:\n')
for study in motivs_dates_targets:
    print(f'- {study}')

In [ ]:
# Metrics
metrics_list = ['m','r','p']

## 1. Data querying <a class="anchor" id="mi"></a>

In [ ]:
# QRP INDEXES query all time in Catalunya
table = 'mitma_qrp'
df_qrp = query_trips_iio_or_qrp_between_dates(table=table, province_groups=province_group)
df_qrp = add_phases_and_motivs(df_qrp, phases_list, motiv_list)

# Describe dataframe
print(f"df_qrp size: {df_qrp.size}")
df_qrp.head(2)

In [ ]:
# P INDEX MATRIX query between 2 dates in Catalunya
table = 'mitma_flux'
df_flux_daily = query_raw_data_or_trips_or_flux_matrix_between_dates(
     table=table, date1=None, date2=None, province_groups=province_group)

# Describe dataframe
print(f"df_flux_daily size: {df_flux_daily.size}")
df_flux_daily.head(3)

## 2. Mobility indexes <a class="anchor" id="mi"></a>

### 2.1. Temporal Series <a class="anchor" id="mi-ts"></a>

In [ ]:
# Daily mobility indexes
for location_target in location_info.keys():
    plot_ts_mqrp_daily_mobility(
        df_qrp, location_target, byweek=False, 
        save_url=f'img/catalunya_analysis/{location_target}/ts_mqrp_daily_{location_target}.png')

In [ ]:
# Weekly mobility indexes
for location_target in location_info.keys():
    plot_ts_mqrp_daily_mobility(
        df_qrp, location_target, byweek=True, 
        save_url=f'img/catalunya_analysis/{location_target}/ts_mqrp_weekly_{location_target}.png')

### 2.2 Maps <a class="anchor" id="mi-maps"></a>

In [ ]:
# Mean daily mobility indexes or 1 day mobility indexes
for location_target in location_info.keys():
    for phase_name, date1, date2 in phases_dates_targets:
            for metric_target in metrics_list:
                plot_map_daily_or_mean_daily_mobility_index(
                    df=df_qrp, target=location_target, metric=metric_target, 
                    phase=phase_name, date1=date1, date2=date2, 
                    save_url=f'img/catalunya_analysis/{location_target}/map_{metric_target}_{phase_name}_phase_{location_target}.png')

### 2.3 Heatmaps <a class="anchor" id="mi-heatmaps"></a>

In [ ]:
# Daily mean mobility index. Y axis = MITMA source region 
for location_target in location_info.keys():
   
    for metric in metrics_list:    
        # Set size of heatmap
        if location_target == 'cat':
            figsize=(12,7)
        if location_target == 'bcn':
            figsize=(13,7)
        if location_target == 'girona':
            figsize=(35,5)
        if location_target == 'tarraco':
            figsize=(20,4)
        if location_target == 'lleida':
            figsize=(25,4)
        
        plot_htmap_mobility_indexes(
            df=df_qrp, target=location_target, metric=metric, y_axis='source', source_by_week=False, figsize=figsize,
            save_url=f'img/catalunya_analysis/{location_target}/heatmap_daily_{metric}_by_source_{location_target}.png')

In [ ]:
# Weekly mean mobility index. Y axis = MITMA source region  
for location_target in location_info.keys():

    for metric in metrics_list:    
        # Set size of heatmap
        if location_target == 'cat':
            figsize=(5,20)
        if location_target == 'bcn':
            figsize=(10,20)
        if location_target == 'girona':
            figsize=(16,12)
        if location_target == 'tarraco':
            figsize=(10,10)
        if location_target == 'lleida':
            figsize=(16,12)
        
        plot_htmap_mobility_indexes(
            df=df_qrp, target=location_target, metric=metric, y_axis='source', source_by_week=True, figsize=figsize,
            save_url=f'img/catalunya_analysis/{location_target}/heatmap_weekly_{metric}_by_source_{location_target}.png')

In [ ]:
# Weekly mean mobility index. Y axis = day of week
figsize = (16, 4)

for location_target in location_info.keys():     
    for metric in metrics_list:
        plot_htmap_mobility_indexes(
            df=df_qrp, target=location_target, metric=metric, y_axis='weekday', source_by_week=True, figsize=figsize,
            save_url=f'img/catalunya_analysis/{location_target}/heatmap_weekly_{metric}_by_weekday_{location_target}.png')

## 2. Δ Mobility indexes <a class="anchor" id="Δmi"></a> 

### 2.1 Temporal Series <a class="anchor" id="Δmi-ts"></a> 

In [ ]:
# Daily delta mobility using phases
compare = 'phase'

for location_target in location_info.keys():
    for metric in metrics_list:
        phase_number = 1
        for phase in all_phases:
            plot_ts_delta_mobility_compared_to_baseline(
                df=df_qrp, metric=metric, loc_target=location_target, compare=compare, baseline=phase, byweek=False,
                save_url=f'img/catalunya_analysis/{location_target}/ts_Δ{metric}_daily_{phase_number}{phase}_{compare}_as_baseline.png')
            phase_number += 1

In [ ]:
# Weekly delta mobility using phases
compare = 'phase'

for location_target in location_info.keys():
    for metric in metrics_list:
        phase_number = 1
        for phase in all_phases:
            plot_ts_delta_mobility_compared_to_baseline(
                df=df_qrp, metric=metric, loc_target=location_target, compare=compare, baseline=phase, byweek=True,
                save_url=f'img/catalunya_analysis/{location_target}/ts_Δ{metric}_weekly_{phase_number}{phase}_{compare}_as_baseline.png')
            phase_number += 1

### 2.2. Maps <a class="anchor" id="Δmi-maps"></a> 

In [ ]:
# MAP Delta mobility in 2 phases
compare = 'phase'
baseline_list = [study_phases[0]] # precovid
target_list = study_phases[1:]

# Plot maps
for location_target in location_info.keys():        
    for metric in metrics_list:                
        for target in target_list:
            for baseline in baseline_list:
                plot_map_delta_mobility_comparing_2_phases_or_motivs_or_days(
                    df_qrp, location_target=location_target, metric=metric, 
                    compare=compare, baseline=baseline, target=target,
                    save_url=f'img/catalunya_analysis/{location_target}/map_Δ{metric}_{compare}_{baseline}_vs_{target}_{location_target}.png')

In [ ]:
# MAP Delta mobility in 2 motivs
compare = 'motiv'
baseline_list = [study_motivs[0]] # baseline
target_list = study_motivs[1:]

# Plot maps
for location_target in location_info.keys():
    for metric in metrics_list:
        for target in target_list:
            for baseline in baseline_list:
                plot_map_delta_mobility_comparing_2_phases_or_motivs_or_days(
                    df_qrp, location_target=location_target, metric=metric, 
                    compare=compare, baseline=baseline, target=target,
                    save_url=f'img/catalunya_analysis/{location_target}/map_Δ{metric}_{compare}_{baseline}_vs_{target}_{location_target}.png')

### 2.3. Heatmaps <a class="anchor" id="Δmi-heatmaps"></a> 

In [ ]:
# Daily delta mobility index comparing 2 phases. Y axis = MITMA source region
compare = 'phase'
baseline_list = [study_phases[0]] # precovid
target_list = study_phases[1:]

for loc_target in location_info.keys():
       
    # Set size of heatmap
    if loc_target == 'cat':
        figsize=(10,60)
    if loc_target == 'bcn':
        figsize=(8,40)
    if loc_target == 'girona':
        figsize=(8,40)
    if loc_target == 'tarraco':
        figsize=(8,40)
    if loc_target == 'lleida':
        figsize=(8,40)
        
    for metric in metrics_list:
        for baseline in baseline_list:
            for target in target_list:
                plot_htmap_delta_mobility_indexes(
                    df=df_qrp, location_target=loc_target, metric=metric, 
                    compare=compare, baseline=baseline, target=target, y_axis='source', source_by_week=False, figsize=figsize,
                    save_url=f'img/catalunya_analysis/{location_target}/heatmap_Δ{metric}_daily_by_source_{compare}_{baseline}_vs_{target}_{location_target}.png')

In [ ]:
# Weekly delta mobility index comparing 2 phases. Y axis = MITMA source region
compare = 'phase'
baseline_list = [study_phases[0]] # precovid
target_list = study_phases[1:]

for loc_target in location_info.keys():
       
    # Set size of heatmap
    if loc_target == 'cat':
        figsize=(10,60)
    if loc_target == 'bcn':
        figsize=(8,40)
    if loc_target == 'girona':
        figsize=(8,40)
    if loc_target == 'tarraco':
        figsize=(8,40)
    if loc_target == 'lleida':
        figsize=(8,40)
        
    for metric in metrics_list:
        for baseline in baseline_list:
            for target in target_list:
                plot_htmap_delta_mobility_indexes(
                    df=df_qrp, location_target=loc_target, metric=metric, 
                    compare=compare, baseline=baseline, target=target, y_axis='source', source_by_week=True, figsize=figsize,
                    save_url=f'img/catalunya_analysis/{location_target}/heatmap_Δ{metric}_weekly_by_source_{compare}_{baseline}_vs_{target}_{location_target}.png')

In [ ]:
# Weekly delta mobility index comparing 2 motivs. Y axis = MITMA source region
compare = 'motiv'
baseline_list = [study_motivs[0]] # baseline
target_list = study_motivs[1:]

for loc_target in location_info.keys():
       
    # Set size of heatmap
    if loc_target == 'cat':
        figsize=(10,60)
    if loc_target == 'bcn':
        figsize=(8,40)
    if loc_target == 'girona':
        figsize=(4,20)
    if loc_target == 'tarraco':
        figsize=(4,20)
    if loc_target == 'lleida':
        figsize=(4,20)
        
    for metric in metrics_list:
        for baseline in baseline_list:
            for target in target_list:
                plot_htmap_delta_mobility_indexes(
                    df=df_qrp, location_target=loc_target, metric=metric, 
                    compare=compare, baseline=baseline, target=target, y_axis='source', source_by_week=True, figsize=figsize,
                    save_url=f'img/catalunya_analysis/{location_target}/heatmap_Δ{metric}_daily_by_source_{compare}_{baseline}_vs_{target}_{location_target}.png')

In [ ]:
# Weekly delta mobility index comparing 2 phases. Y axis = weekday
compare = 'phase'
baseline_list = [study_phases[0]] # baseline
target_list = ['all_time']

for loc_target in list(location_info.keys()):     
    for metric in metrics_list:
        for baseline in baseline_list:
            for target in target_list:
                plot_htmap_delta_mobility_indexes(
                    df=df_qrp, location_target=loc_target, metric=metric, 
                    compare=compare, baseline=baseline, target=target, y_axis='weekday', source_by_week=True, figsize=(16, 4),
                    save_url=f'img/catalunya_analysis/{loc_target}/heatmap_Δ{metric}_weekly_by_weekday_{compare}_{baseline}_vs_{target}_{loc_target}.png')

In [ ]:
# Weekly delta mobility index comparing 2 motivs. Y axis = weekday
compare = 'motiv'
baseline_list = [study_motivs[0]] # baseline
target_list = study_motivs[1:]

for loc_target in location_info.keys():     
    for metric in metrics_list:
        for baseline in baseline_list:
            for target in target_list:
                plot_htmap_delta_mobility_indexes(
                    df=df_qrp, location_target=loc_target, metric=metric, 
                    compare=compare, baseline=baseline, target=target, y_axis='weekday', source_by_week=True, figsize=(16, 4),
                    save_url=f'img/catalunya_analysis/{location_target}/heatmap_Δ{metric}_weekly_by_weekday_{compare}_{baseline}_vs_{target}_{location_target}.png')